In [1]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
print("✓ Loaded PhoBERT tokenizer")

e:\CHATBOT MODEL\training\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Loaded PhoBERT tokenizer


## Test với từ "sốt"

In [2]:
test_text = "sốt"

# Tokenize
tokens = tokenizer.tokenize(test_text)
input_ids = tokenizer.encode(test_text, add_special_tokens=True)
decoded_tokens = tokenizer.convert_ids_to_tokens(input_ids)

print(f"Text: {test_text}")
print(f"Tokens: {tokens}")
print(f"Input IDs: {input_ids}")
print(f"Decoded tokens: {decoded_tokens}")
print(f"\nVấn đề: Từ 'sốt' bị tách thành: {decoded_tokens[1:-1]}")

Text: sốt
Tokens: ['sốt']
Input IDs: [0, 2122, 2]
Decoded tokens: ['<s>', 'sốt', '</s>']

Vấn đề: Từ 'sốt' bị tách thành: ['sốt']


## Test với câu hoàn chỉnh

In [3]:
test_sentence = "Tôi bị sốt cao và đau đầu"

tokens = tokenizer.tokenize(test_sentence)
input_ids = tokenizer.encode(test_sentence, add_special_tokens=True)
decoded_tokens = tokenizer.convert_ids_to_tokens(input_ids)

print(f"Câu: {test_sentence}")
print(f"\nTokens:")
for i, token in enumerate(decoded_tokens):
    print(f"  {i}: {token}")

print(f"\nChú ý: Các token có '@@' là subword tokens (phần tiếp theo của từ trước đó)")

Câu: Tôi bị sốt cao và đau đầu

Tokens:
  0: <s>
  1: Tôi
  2: bị
  3: sốt
  4: cao
  5: và
  6: đau
  7: đầu
  8: </s>

Chú ý: Các token có '@@' là subword tokens (phần tiếp theo của từ trước đó)


## Giải thích vấn đề

**PhoBERT sử dụng BPE (Byte-Pair Encoding):**
- Từ được tách thành các subword
- `@@` đánh dấu phần tiếp theo của từ (không phải đầu từ)
- Ví dụ: `sốt` → `['s', '@@ố', '@@t']`

**Vấn đề trong code:**
- Khi in kết quả NER, bạn in trực tiếp tokens từ tokenizer
- Cần ghép lại các subword thành từ gốc

**Giải pháp:**
1. Loại bỏ `@@` khi hiển thị
2. Ghép các subword lại thành từ hoàn chỉnh
3. Chỉ lấy label của token đầu tiên trong mỗi từ

## Code sửa lỗi hiển thị

In [4]:
def decode_tokens_and_labels(tokens, labels):
    """
    Ghép subword tokens thành từ gốc và lấy label tương ứng
    """
    words = []
    word_labels = []
    current_word = ""
    current_label = None
    
    for token, label in zip(tokens, labels):
        # Bỏ qua special tokens
        if token in ['<s>', '</s>', '<pad>']:
            continue
        
        # Subword token (tiếp tục từ trước)
        if token.startswith('@@'):
            current_word += token[2:]  # Bỏ @@
        # Token mới (bắt đầu từ mới)
        else:
            # Lưu từ trước (nếu có)
            if current_word:
                words.append(current_word)
                word_labels.append(current_label)
            
            # Bắt đầu từ mới
            current_word = token
            current_label = label
    
    # Lưu từ cuối cùng
    if current_word:
        words.append(current_word)
        word_labels.append(current_label)
    
    return words, word_labels

# Test
test_tokens = ['<s>', 'Tôi', 'bị', 's', '@@ố', '@@t', 'cao', '</s>']
test_labels = ['O', 'O', 'O', 'B-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM', 'O', 'O']

words, labels = decode_tokens_and_labels(test_tokens, test_labels)

print("Kết quả sau khi decode:")
for word, label in zip(words, labels):
    print(f"  {word:15s} -> {label}")

print("\n✅ Đã ghép 's' + '@@ố' + '@@t' = 'sốt'")

Kết quả sau khi decode:
  Tôi             -> O
  bị              -> O
  sốt             -> B-SYMPTOM
  cao             -> O

✅ Đã ghép 's' + '@@ố' + '@@t' = 'sốt'


## Cập nhật code test trong notebook training

In [6]:
# Code cũ (SAI):
print("\n❌ Cách hiển thị cũ (sai):")
test_tokens = ['<s>', 'Tôi', 'bị', 's', '@@ố', '@@t', 'cao', '</s>']
test_labels = ['O', 'O', 'O', 'B-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM', 'O', 'O']

for token, label in zip(test_tokens, test_labels):
    if token not in ['<s>', '</s>', '<pad>']:
        print(f"  {token:20s} -> {label}")

# Code mới (ĐÚNG):
print("\n✅ Cách hiển thị mới (đúng):")
words, word_labels = decode_tokens_and_labels(test_tokens, test_labels)
for word, label in zip(words, word_labels):
    print(f"  {word:20s} -> {label}")


❌ Cách hiển thị cũ (sai):
  Tôi                  -> O
  bị                   -> O
  s                    -> B-SYMPTOM
  @@ố                  -> I-SYMPTOM
  @@t                  -> I-SYMPTOM
  cao                  -> O

✅ Cách hiển thị mới (đúng):
  Tôi                  -> O
  bị                   -> O
  sốt                  -> B-SYMPTOM
  cao                  -> O
